In [1]:
# Initial imports
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
# Load the ventas_linea.csv dataset.
ventas_df = pd.read_csv('./almacenes_ventas.csv')
ventas_df

,product_id,description,date,name,qty,price_unit
0,26078,XIAOMI REDMI-NOTE-11S 6/128 NGO,2023-02-24,ALLENDE (QRO),1,4568.110000
1,26579,MOTO G41 4/128 DORADO,2023-02-24,ALLENDE (QRO),1,3620.680000
2,26579,MOTO G41 4/128 DORADO,2023-02-24,ALLENDE (QRO),1,3619.827500
3,26585,MOTO G22 4/128 NEGRO,2023-02-24,PLAZA MAYOR (LEON),1,3878.450000
4,25565,MOTO G60S 6/128 AZUL,2023-02-24,MADERO (LEON),1,4481.896552
...,...,...,...,...,...,...
34097,26053,ZTE L8 1/32 NEGRO,2023-02-24,CORREGIDORA 3 (QRO),1,1033.620000
34098,31000,APPLE IPHONE-11 4/64 BLANCO KIT,2023-02-24,PLAZA PATIO (QRO),1,9481.900000
34099,26578,MOTO G41 4/128 NEGRO,2023-02-24,ALLENDE (QRO),1,3275.000000
34100,25533,MOTO E20 2/32 AZUL,2023-02-24,PLAZA PATIO (QRO),1,1723.280000


In [3]:

# Convert the date column to datetime and set it as the index
ventas_df['date'] = pd.to_datetime(ventas_df['date'])
ventas_df.set_index('date', inplace=True)

# Split the 'description' column into multiple columns
ventas_df[['brand', 'model', 'details']] = ventas_df['description'].str.split(' ', n=2, expand=True)

# Compute the total sales column
ventas_df['total_sales'] = ventas_df['qty'] * ventas_df['price_unit']

# Group the data frame by name and month, and sum the qty and total sales columns
final_ventas_df = ventas_df.groupby([pd.Grouper(freq='M'), 'name', 'brand']).agg({'qty': 'sum', 'price_unit':'mean', 'total_sales': 'sum'})

# Reset the index to make the month, name, and description columns
final_ventas_df = final_ventas_df.reset_index()

# Format the month column as month names
final_ventas_df['date'] = final_ventas_df['date'].dt.strftime('%B')

# Rename the columns
final_ventas_df = final_ventas_df.rename(columns={'date': 'month'})

# Set the display option to show numbers without scientific notation
pd.options.display.float_format = '{:,.2f}'.format

# Print the summary data frame
final_ventas_df = final_ventas_df[['name', 'brand', 'month', 'qty', 'price_unit', 'total_sales']].sort_values('total_sales', ascending=False)
final_ventas_df

,name,brand,month,qty,price_unit,total_sales
2547,ALLENDE (QRO),SAM,December,253,"3,575.06","904,489.00"
2911,ALLENDE (QRO),SAM,January,237,"3,510.55","832,000.06"
2227,ALLENDE (QRO),SAM,November,190,"3,891.20","741,901.21"
1629,ALLENDE (QRO),SAM,September,182,"4,025.92","729,146.39"
326,ALLENDE (QRO),SAM,May,140,"4,572.41","640,137.86"
...,...,...,...,...,...,...
1223,PARQUE (QRO),I-MOBIL,July,1,240.52,240.52
938,ALAMEDA (QRO),I-MOBIL,July,1,240.52,240.52
553,PARQUE (CELAYA),I-MOBIL,May,1,240.52,240.52
1348,BOULEVARES (QRO),ROUTER,August,1,0.01,0.01


In [4]:
# Export the new DataFrame to a CSV file
final_ventas_df.to_csv('final_ventas.csv', index=False)